# Creating template files

In [ ]:
!tar -zxvf template.tar.gz

In [ ]:
!plink --bfile template/template --recode --tab --out template/template --noweb

In [ ]:
!cut -f 7- template/template.ped >> template/cut_template.ped

# Creating .bed files

In [ ]:
import pandas as pd 

cytokines = pd.read_excel("Sepsis_patient_cytokine_levels.xlsx")

In [ ]:
mean_cytokine = cytokines.mean(0)

In [ ]:
ped = pd.read_csv("template/template.fam", sep=' ', 
                  names=["fid", "iid", "father", "mother", "gender", "trait"])

In [ ]:
import math
import os
import subprocess

cwd = os.getcwd()

for cytokine in mean_cytokine.iteritems():
    destination = os.path.join(cwd, 'binary_cytokines/'+cytokine[0])
    for index,row in ped.iterrows():
        cytokine_level = float(cytokines.loc[cytokines['SampleNumber'] == row['fid']][cytokine[0]].values)
        if math.isnan(cytokine_level):
            ped.drop(ped.index[index])
        elif cytokine_level < mean_cytokine[cytokine[0]]:
            ped.at[index, 'trait'] = 1
        else:
            ped.at[index, 'trait'] = 2
    if not os.path.exists(destination):
        os.makedirs(destination)
    ped.to_csv(os.path.join(destination, cytokine[0]+"_temp.ped"), 
               header=None, index=None, sep=' ', mode='a')
    subprocess.call(['cp', 'template/template.map', destination])
    subprocess.call(['mv', os.path.join(destination, 'template.map'), os.path.join(destination, cytokine[0]+".map")])
    paste = "paste -d' ' %s template/cut_template.ped >> %s" % (os.path.join(destination, cytokine[0]+"_temp.ped"), os.path.join(destination, cytokine[0]+".ped"))
    !{paste}
    bed = "plink --file %s --make-bed --out %s" % (os.path.join(destination, cytokine[0]), os.path.join(destination, cytokine[0]))
    gwas = "plink --bfile %s --assoc --out %s --noweb" % (os.path.join(desination, cytokine[0], os.path.join(destination, cytokine[0])
    !{bed}
    !{gwas}
    subprocess.call(['Rscript', os.path.join(destination, cytokine[0]+".assoc"), os.path.join(destination, cytokine[0]+".png")])
    